In [ ]:
from google.colab import files
import pandas as pd


uploaded = files.upload()


df = pd.read_csv("adolescent_health_data.csv")


df.head()


Saving adolescent_health_data.csv to adolescent_health_data.csv


,OBS,CITY,CTYPE,CTYPE_SD,SCHOOL,STYPE,STRATA,STRATA_NM,CLUSTER,GROUP,...,E_LT_F,E_LT_SF,E_LT_M,E_LT_SM,E_EDU_F,E_KRN_F,E_BORN_F,E_EDU_M,E_KRN_M,E_BORN_M
0,A100001,서울,대도시,대도시,중학교,남녀공학,2024_64,서울대_중학교4_3,1.0,서울대4-3,...,1.0,9999.0,1.0,9999.0,3.0,1.0,9999.0,3.0,1.0,9999.0
1,A100002,서울,대도시,대도시,중학교,남녀공학,2024_64,서울대_중학교4_3,1.0,서울대4-3,...,1.0,9999.0,1.0,9999.0,3.0,1.0,9999.0,3.0,1.0,9999.0
2,A100003,서울,대도시,대도시,중학교,남녀공학,2024_64,서울대_중학교4_3,1.0,서울대4-3,...,1.0,9999.0,1.0,9999.0,3.0,1.0,9999.0,3.0,1.0,9999.0
3,A100004,서울,대도시,대도시,중학교,남녀공학,2024_64,서울대_중학교4_3,1.0,서울대4-3,...,2.0,9999.0,1.0,9999.0,3.0,1.0,9999.0,3.0,1.0,9999.0
4,A100005,서울,대도시,대도시,중학교,남녀공학,2024_64,서울대_중학교4_3,1.0,서울대4-3,...,1.0,9999.0,1.0,9999.0,3.0,1.0,9999.0,3.0,1.0,9999.0


In [ ]:
import pandas as pd

df = pd.read_csv("adolescent_health_data.csv")

invalid_wk = df[
    ((df['M_WK_HR'].isna()) | (df['M_WK_HR'] == '')) &
    ((df['M_WK_MM'].isna()) | (df['M_WK_MM'] == ''))
]

invalid_wk_excel_rows = invalid_wk.index + 2

print("M_WK_HR, M_WK_MM 둘 다 공백/NaN인 행 개수:", len(invalid_wk))
print("엑셀에서의 행 번호 (앞 20개):", invalid_wk_excel_rows[:20].tolist())


M_WK_HR, M_WK_MM 둘 다 공백/NaN인 행 개수: 4140
엑셀에서의 행 번호 (앞 20개): [30, 33, 36, 43, 54, 58, 61, 70, 71, 120, 121, 126, 129, 130, 134, 146, 152, 160, 166, 174]


In [ ]:
import pandas as pd
import numpy as np

# CSV 불러오기
df = pd.read_csv("adolescent_health_data.csv")

# 공백을 NaN으로 변환
df['M_WK_HR'] = df['M_WK_HR'].replace('', np.nan)
df['M_WK_MM'] = df['M_WK_MM'].replace('', np.nan)

# 숫자로 변환
df['M_WK_HR'] = pd.to_numeric(df['M_WK_HR'], errors='coerce')
df['M_WK_MM'] = pd.to_numeric(df['M_WK_MM'], errors='coerce')

# 그룹 정의 (성별, 학년, 도시)
group_cols = ['SEX', 'GRADE', 'CITY']

# 그룹별 중앙값 계산
group_medians = df.groupby(group_cols)[['M_WK_HR', 'M_WK_MM']].median()

# 중앙값을 기준으로 결측치 채우기
def fill_with_group_median(row):
    if pd.isna(row['M_WK_HR']) and pd.isna(row['M_WK_MM']):
        medians = group_medians.loc[(row['SEX'], row['GRADE'], row['CITY'])]
        row['M_WK_HR'] = medians['M_WK_HR']
        row['M_WK_MM'] = medians['M_WK_MM']
    return row

df = df.apply(fill_with_group_median, axis=1)

# 결과 확인
print(df[['SEX', 'GRADE', 'CITY', 'M_WK_HR', 'M_WK_MM']].head())



   SEX  GRADE CITY  M_WK_HR  M_WK_MM
0  2.0    1.0   서울      8.0      0.0
1  2.0    1.0   서울      7.0     10.0
2  2.0    1.0   서울      6.0     30.0
3  2.0    1.0   서울      6.0     10.0
4  1.0    1.0   서울      8.0      0.0


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("adolescent_health_data.csv")

# 숫자형 변환
df["INT_SPWD_TM"] = pd.to_numeric(df["INT_SPWD_TM"], errors="coerce")

# 시/분 분해
df["INT_SPWD_HR"] = (df["INT_SPWD_TM"] // 60).astype("Int64")
df["INT_SPWD_MM"] = (df["INT_SPWD_TM"] % 60).astype("Int64")

# "HH:MM" 문자열도 함께 생성
df["INT_SPWD_HHMM"] = df.apply(
    lambda r: f"{int(r['INT_SPWD_HR']):02d}:{int(r['INT_SPWD_MM']):02d}"
              if pd.notna(r["INT_SPWD_TM"]) else pd.NA,
    axis=1
)

# 예시 확인
print(df[["INT_SPWD_TM","INT_SPWD_HR","INT_SPWD_MM","INT_SPWD_HHMM"]].head(10))


   INT_SPWD_TM  INT_SPWD_HR  INT_SPWD_MM INT_SPWD_HHMM
0        210.0            3           30         03:30
1        120.0            2            0         02:00
2          NaN         <NA>         <NA>          <NA>
3        510.0            8           30         08:30
4         60.0            1            0         01:00
5        270.0            4           30         04:30
6        300.0            5            0         05:00
7          NaN         <NA>         <NA>          <NA>
8        320.0            5           20         05:20
9        120.0            2            0         02:00


In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display

# ===== 0) 데이터 불러오기 =====
PATH = "adolescent_health_data.csv"  # 경로 맞춰서 수정
df = pd.read_csv(PATH, low_memory=False)
df = df.replace([9999, 8888, 7777], np.nan)  # 특수 결측 통일

pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 2000)

# ===== 1) Q번호 → 실제 변수명 매핑 =====
COL = {
    "Q54": "TC_LT",
    "Q55": "TC_FAGE",
    "Q56": "TC_DAYS",
    "Q57": "TC_DAGE",
    "Q58": "TC_AMNT",
    "Q59": "TC_EC_LT",
    "Q60": "TC_EC_FAGE",
    "Q61": "TC_EC_MN",
    "Q62": "TC_EC_DAGE",
    "Q63": "TC_HTP_LT",
    "Q64": "TC_HTP_FAGE",
    "Q65": "TC_HTP_MN",
    "Q66": "TC_HTP_DAGE",
    "Q67": "TC_HTP_AMNT",
    "Q68": "TC_PLC",
    "Q69": "TC_GET_M",
    "Q70": "TC_ACCESS",
    "Q71": "TC_QT_YR",
}
def C(q):
    col = COL.get(q)
    return col if (col in df.columns) else None

# ===== 2) 규칙(필수/금지/부모결측) 정의 =====

# A) 필수
REQ_RULES = [
  ("R01", "TC_LT==1 → TC_EC_LT 필수",                 ("Q54","==1"),  ["Q59"]),
  ("R02", "TC_LT==2 → TC_FAGE 필수",                  ("Q54","==2"),  ["Q55"]),
  ("R03", "TC_LT==2 → TC_DAYS 필수",                  ("Q54","==2"),  ["Q56"]),
  ("R04", "TC_DAYS==1 → TC_EC_LT 필수",               ("Q56","==1"),  ["Q59"]),
  ("R05", "TC_DAYS in 2~6 → TC_AMNT 필수",            ("Q56","in[2,3,4,5,6]"), ["Q58"]),
  ("R06", "TC_DAYS==7 → TC_DAGE 필수",                ("Q56","==7"),  ["Q57"]),
  ("R07", "TC_EC_LT==1 → TC_HTP_LT 필수",             ("Q59","==1"),  ["Q63"]),
  ("R08", "TC_EC_LT==2 → TC_EC_FAGE 필수",            ("Q59","==2"),  ["Q60"]),
  ("R09", "TC_EC_LT==2 → TC_EC_MN 필수",              ("Q59","==2"),  ["Q61"]),
  ("R10", "TC_EC_MN in 1~6 → TC_HTP_LT 필수",         ("Q61","in[1,2,3,4,5,6]"), ["Q63"]),
  ("R11", "TC_EC_MN==7 → TC_EC_DAGE 필수",            ("Q61","==7"),  ["Q62"]),
  ("R12", "TC_HTP_LT==1 → TC_ACCESS 필수",            ("Q63","==1"),  ["Q70"]),
  ("R13", "TC_HTP_LT==2 → TC_HTP_FAGE 필수",          ("Q63","==2"),  ["Q64"]),
  ("R14", "TC_HTP_LT==2 → TC_HTP_MN 필수",            ("Q63","==2"),  ["Q65"]),
  ("R15", "TC_HTP_MN==1 → TC_ACCESS 필수",            ("Q65","==1"),  ["Q70"]),
  ("R16", "TC_HTP_MN in 2~6 → TC_HTP_AMNT 필수",      ("Q65","in[2,3,4,5,6]"), ["Q67"]),
  ("R17", "TC_HTP_MN==7 → TC_HTP_DAGE 필수",          ("Q65","==7"),  ["Q66"]),
  ("R18", "(TC_DAYS or TC_EC_MN or TC_HTP_MN) in 2~7 → TC_PLC,TC_GET_M,TC_QT_YR 필수",
          ("ANY","inrange[2..7]"), ["Q68","Q69","Q71"]),
]

# B) 금지
FORBID_RULES = [
  ("F01", "TC_LT==1 → TC_FAGE,TC_DAYS 금지",                ("Q54","==1"),  ["Q55","Q56"]),
  ("F02", "TC_DAYS==1 → TC_DAGE,TC_AMNT 금지",              ("Q56","==1"),  ["Q57","Q58"]),
  ("F03", "TC_DAYS in 2~6 → TC_DAGE,TC_EC_LT 금지",         ("Q56","in[2,3,4,5,6]"), ["Q57","Q59"]),
  ("F04", "TC_DAYS==7 → TC_AMNT,TC_EC_LT 금지",             ("Q56","==7"),  ["Q58","Q59"]),
  ("F05", "TC_EC_LT==1 → TC_EC_FAGE,TC_EC_MN,TC_EC_DAGE 금지", ("Q59","==1"), ["Q60","Q61","Q62"]),
  ("F06", "TC_EC_MN in 1~6 → TC_EC_DAGE 금지",              ("Q61","in[1,2,3,4,5,6]"), ["Q62"]),
  ("F07", "TC_HTP_LT==1 → TC_HTP_FAGE,TC_HTP_MN,TC_HTP_DAGE,TC_HTP_AMNT 금지",
          ("Q63","==1"), ["Q64","Q65","Q66","Q67"]),  # 68/69/71은 집합 규칙으로 따로 관리
  ("F08", "TC_HTP_MN==1 → TC_HTP_DAGE,TC_HTP_AMNT 금지",    ("Q65","==1"),  ["Q66","Q67"]),
  ("F09", "TC_HTP_MN in 2~6 → TC_HTP_DAGE 금지",            ("Q65","in[2,3,4,5,6]"), ["Q66"]),
  ("F10", "TC_HTP_MN==7 → TC_HTP_AMNT 금지",                ("Q65","==7"),  ["Q67"]),
  ("F11", "NOT (TC_DAYS|TC_EC_MN|TC_HTP_MN in 2~7) → TC_PLC,TC_GET_M,TC_QT_YR 금지",
          ("ANY_NONE","out_of_2to7"), ["Q68","Q69","Q71"]),
]

# C) 부모 결측(게이트 결측)
GATE_CHILDREN = {
    "Q54": ["Q55","Q56","Q59"],
    "Q56": ["Q57","Q58","Q59"],
    "Q59": ["Q60","Q61","Q62","Q63"],
    "Q61": ["Q62","Q63"],
    "Q63": ["Q64","Q65","Q66","Q67","Q68","Q69","Q70","Q71"],
    "Q65": ["Q66","Q67","Q70"],
}
COMPOSITE_PARENTS = ["Q56","Q61","Q65"]
COMPOSITE_CHILDREN = ["Q68","Q69","Q71"]

# ===== 3) 유틸 =====
def m_cond(series, cond: str):
    s = pd.to_numeric(series, errors="coerce")
    if cond.startswith("=="):
        v = int(cond.replace("==",""))
        return s == v
    if cond.startswith("in[") and "," in cond:
        vals = [int(x) for x in cond[3:-1].split(",")]
        return s.isin(vals)
    if cond.startswith("inrange["):
        lo, hi = cond[8:-1].split("..")
        return s.between(int(lo), int(hi))
    raise ValueError(f"Unknown cond: {cond}")

# ===== 4) 검사 함수 =====
def check_required_missing(df):
    out = []
    # 집합 규칙(ANY)
    any_2to7 = pd.Series(False, index=df.index)
    parts = []
    for q in ["Q56","Q61","Q65"]:
        col = C(q)
        if col: parts.append(pd.to_numeric(df[col], errors="coerce").between(2,7))
    if parts:
        any_2to7 = parts[0]
        for p in parts[1:]:
            any_2to7 = any_2to7 | p

    for rid, desc, (gate, cond), targets in REQ_RULES:
        tcols = [C(t) for t in targets if C(t)]
        if not tcols:
            continue
        if gate == "ANY":
            mask_gate = any_2to7
            gate_col, gate_val = "(composite)", "(2~7 in any of TC_DAYS/TC_EC_MN/TC_HTP_MN)"
        else:
            gate_col = C(gate)
            if not gate_col:
                continue
            mask_gate = m_cond(df[gate_col], cond)
            gate_val = cond

        miss = pd.Series(False, index=df.index)
        for t in tcols:
            miss = miss | df[t].isna()

        idx = df.index[mask_gate & miss]
        if len(idx)==0:
            continue
        view_cols = [c for c in [gate_col]+tcols if c and c in df.columns]
        tmp = df.loc[idx, view_cols].copy()
        tmp["_viol_cols"] = tmp[tcols].isna().apply(lambda r: ",".join([c for c in tcols if pd.isna(r[c])]), axis=1)
        tmp.insert(0, "_expectation", f"must be NON-NULL in [{', '.join(tcols)}]")
        tmp.insert(0, "_condition", f"{gate_col} {gate_val}")
        tmp.insert(0, "_type", "RequiredMissing")
        tmp.insert(0, "_rule", rid)
        tmp.insert(0, "_desc", desc)
        out.append(tmp.reset_index().rename(columns={"index":"_row"}))
    return pd.concat(out, ignore_index=True) if out else pd.DataFrame()

def check_forbidden_filled(df):
    out = []
    # 집합 반대(ANY_NONE)
    none_2to7 = pd.Series(True, index=df.index)
    parts = []
    for q in ["Q56","Q61","Q65"]:
        col = C(q)
        if col: parts.append(~pd.to_numeric(df[col], errors="coerce").between(2,7))
    if parts:
        none_2to7 = parts[0]
        for p in parts[1:]:
            none_2to7 = none_2to7 & p

    for rid, desc, (gate, cond), forbids in FORBID_RULES:
        fcols = [C(t) for t in forbids if C(t)]
        if not fcols:
            continue
        if gate == "ANY_NONE":
            mask_gate = none_2to7
            gate_col, gate_val = "(composite)", "(NOT in 2~7 for ALL of TC_DAYS/TC_EC_MN/TC_HTP_MN)"
        else:
            gate_col = C(gate)
            if not gate_col:
                continue
            mask_gate = m_cond(df[gate_col], cond)
            gate_val = cond

        bad = pd.Series(False, index=df.index)
        for t in fcols:
            bad = bad | df[t].notna()

        idx = df.index[mask_gate & bad]
        if len(idx)==0:
            continue
        view_cols = [c for c in [gate_col]+fcols if c and c in df.columns]
        tmp = df.loc[idx, view_cols].copy()
        tmp["_viol_cols"] = tmp[fcols].notna().apply(lambda r: ",".join([c for c in fcols if pd.notna(r[c])]), axis=1)
        tmp.insert(0, "_expectation", f"must be NULL in [{', '.join(fcols)}]")
        tmp.insert(0, "_condition", f"{gate_col} {gate_val}")
        tmp.insert(0, "_type", "ForbiddenFilled")
        tmp.insert(0, "_rule", rid)
        tmp.insert(0, "_desc", desc)
        out.append(tmp.reset_index().rename(columns={"index":"_row"}))
    return pd.concat(out, ignore_index=True) if out else pd.DataFrame()

def check_gate_missing(df):
    out = []
    # 단일 부모
    for parent, children in GATE_CHILDREN.items():
        pcol = C(parent)
        if not pcol:
            continue
        ccols = [C(c) for c in children if C(c)]
        if not ccols:
            continue
        parent_nan = df[pcol].isna()
        child_filled = pd.Series(False, index=df.index)
        for c in ccols:
            child_filled = child_filled | df[c].notna()
        idx = df.index[parent_nan & child_filled]
        if len(idx)==0:
            continue
        view_cols = [pcol] + ccols
        tmp = df.loc[idx, view_cols].copy()
        tmp["_viol_cols"] = tmp[ccols].notna().apply(lambda r: ",".join([c for c in ccols if pd.notna(r[c])]), axis=1)
        tmp.insert(0, "_expectation", f"parent {pcol} is NON-NULL before children [{', '.join(ccols)}]")
        tmp.insert(0, "_condition", f"{pcol} is NaN")
        tmp.insert(0, "_type", "GateMissing")
        tmp.insert(0, "_rule", f"G_{parent}")
        tmp.insert(0, "_desc", f"{parent} missing but children present")
        out.append(tmp.reset_index().rename(columns={"index":"_row"}))

    # 집합 부모 모두 NaN인데 68/69/71 값 존재
    parents = [C(p) for p in COMPOSITE_PARENTS if C(p)]
    children = [C(c) for c in COMPOSITE_CHILDREN if C(c)]
    if parents and children:
        all_parents_nan = pd.Series(True, index=df.index)
        for p in parents:
            all_parents_nan = all_parents_nan & df[p].isna()
        any_child_filled = pd.Series(False, index=df.index)
        for ch in children:
            any_child_filled = any_child_filled | df[ch].notna()
        idx = df.index[all_parents_nan & any_child_filled]
        if len(idx):
            view_cols = parents + children
            tmp = df.loc[idx, view_cols].copy()
            tmp["_viol_cols"] = tmp[children].notna().apply(lambda r: ",".join([c for c in children if pd.notna(r[c])]), axis=1)
            tmp.insert(0, "_expectation", f"parents {', '.join(parents)} should be NON-NULL before children [{', '.join(children)}]")
            tmp.insert(0, "_condition", f"all parents NaN: {', '.join(parents)}")
            tmp.insert(0, "_type", "GateMissing")
            tmp.insert(0, "_rule", "G_COMPOSITE")
            tmp.insert(0, "_desc", "Composite parents missing but children present")
            out.append(tmp.reset_index().rename(columns={"index":"_row"}))
    return pd.concat(out, ignore_index=True) if out else pd.DataFrame()

# ===== 5) 실행 & 표로 확인 =====
req_df  = check_required_missing(df)
forb_df = check_forbidden_filled(df)
gate_df = check_gate_missing(df)

print("▶ Required Missing (필수인데 비어 있음):", len(req_df));   display(req_df.head(10))
print("\n▶ Forbidden Filled (비워야 하는데 값이 있음):", len(forb_df)); display(forb_df.head(10))
print("\n▶ Gate Missing (부모 NaN인데 자식 값 존재):", len(gate_df));   display(gate_df.head(10))

combined = pd.concat([
    req_df.assign(_group="RequiredMissing"),
    forb_df.assign(_group="ForbiddenFilled"),
    gate_df.assign(_group="GateMissing")
], ignore_index=True)

print("\n총 오류(결측/논리) 개수:", len(combined))
display(combined.head(20))

# (선택) 규칙별 건수 요약
if len(combined):
    summary = (combined
               .groupby(["_group","_rule","_desc","_condition","_expectation"], dropna=False)
               .size().reset_index(name="count")
               .sort_values(["_group","count"], ascending=[True, False]))
    print("\n규칙별 건수 요약:")
    display(summary.head(50))


▶ Required Missing (필수인데 비어 있음): 219


,_row,_desc,_rule,_type,_condition,_expectation,TC_LT,TC_FAGE,_viol_cols,TC_DAYS,TC_DAGE,TC_EC_LT,TC_EC_FAGE,TC_EC_MN,TC_EC_DAGE,TC_HTP_LT,TC_HTP_FAGE,TC_HTP_MN,TC_HTP_DAGE
0,603,TC_LT==2 → TC_FAGE 필수,R02,RequiredMissing,TC_LT ==2,must be NON-NULL in [TC_FAGE],2.0,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,792,TC_LT==2 → TC_FAGE 필수,R02,RequiredMissing,TC_LT ==2,must be NON-NULL in [TC_FAGE],2.0,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1767,TC_LT==2 → TC_FAGE 필수,R02,RequiredMissing,TC_LT ==2,must be NON-NULL in [TC_FAGE],2.0,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2028,TC_LT==2 → TC_FAGE 필수,R02,RequiredMissing,TC_LT ==2,must be NON-NULL in [TC_FAGE],2.0,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2338,TC_LT==2 → TC_FAGE 필수,R02,RequiredMissing,TC_LT ==2,must be NON-NULL in [TC_FAGE],2.0,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3244,TC_LT==2 → TC_FAGE 필수,R02,RequiredMissing,TC_LT ==2,must be NON-NULL in [TC_FAGE],2.0,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3796,TC_LT==2 → TC_FAGE 필수,R02,RequiredMissing,TC_LT ==2,must be NON-NULL in [TC_FAGE],2.0,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4249,TC_LT==2 → TC_FAGE 필수,R02,RequiredMissing,TC_LT ==2,must be NON-NULL in [TC_FAGE],2.0,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4427,TC_LT==2 → TC_FAGE 필수,R02,RequiredMissing,TC_LT ==2,must be NON-NULL in [TC_FAGE],2.0,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,4491,TC_LT==2 → TC_FAGE 필수,R02,RequiredMissing,TC_LT ==2,must be NON-NULL in [TC_FAGE],2.0,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



▶ Forbidden Filled (비워야 하는데 값이 있음): 2136


,_row,_desc,_rule,_type,_condition,_expectation,TC_DAYS,TC_DAGE,TC_EC_LT,_viol_cols,TC_AMNT,TC_HTP_MN,TC_HTP_AMNT,TC_PLC,TC_GET_M,TC_QT_YR
0,414,"TC_DAYS in 2~6 → TC_DAGE,TC_EC_LT 금지",F03,ForbiddenFilled,"TC_DAYS in[2,3,4,5,6]","must be NULL in [TC_DAGE, TC_EC_LT]",6.0,NaN,2.0,"TC_DAGE,TC_EC_LT",NaN,NaN,NaN,NaN,NaN,NaN
1,444,"TC_DAYS in 2~6 → TC_DAGE,TC_EC_LT 금지",F03,ForbiddenFilled,"TC_DAYS in[2,3,4,5,6]","must be NULL in [TC_DAGE, TC_EC_LT]",2.0,NaN,2.0,"TC_DAGE,TC_EC_LT",NaN,NaN,NaN,NaN,NaN,NaN
2,490,"TC_DAYS in 2~6 → TC_DAGE,TC_EC_LT 금지",F03,ForbiddenFilled,"TC_DAYS in[2,3,4,5,6]","must be NULL in [TC_DAGE, TC_EC_LT]",2.0,NaN,2.0,"TC_DAGE,TC_EC_LT",NaN,NaN,NaN,NaN,NaN,NaN
3,592,"TC_DAYS in 2~6 → TC_DAGE,TC_EC_LT 금지",F03,ForbiddenFilled,"TC_DAYS in[2,3,4,5,6]","must be NULL in [TC_DAGE, TC_EC_LT]",4.0,NaN,2.0,"TC_DAGE,TC_EC_LT",NaN,NaN,NaN,NaN,NaN,NaN
4,594,"TC_DAYS in 2~6 → TC_DAGE,TC_EC_LT 금지",F03,ForbiddenFilled,"TC_DAYS in[2,3,4,5,6]","must be NULL in [TC_DAGE, TC_EC_LT]",3.0,NaN,1.0,"TC_DAGE,TC_EC_LT",NaN,NaN,NaN,NaN,NaN,NaN
5,636,"TC_DAYS in 2~6 → TC_DAGE,TC_EC_LT 금지",F03,ForbiddenFilled,"TC_DAYS in[2,3,4,5,6]","must be NULL in [TC_DAGE, TC_EC_LT]",5.0,NaN,2.0,"TC_DAGE,TC_EC_LT",NaN,NaN,NaN,NaN,NaN,NaN
6,659,"TC_DAYS in 2~6 → TC_DAGE,TC_EC_LT 금지",F03,ForbiddenFilled,"TC_DAYS in[2,3,4,5,6]","must be NULL in [TC_DAGE, TC_EC_LT]",6.0,NaN,2.0,"TC_DAGE,TC_EC_LT",NaN,NaN,NaN,NaN,NaN,NaN
7,1006,"TC_DAYS in 2~6 → TC_DAGE,TC_EC_LT 금지",F03,ForbiddenFilled,"TC_DAYS in[2,3,4,5,6]","must be NULL in [TC_DAGE, TC_EC_LT]",5.0,NaN,2.0,"TC_DAGE,TC_EC_LT",NaN,NaN,NaN,NaN,NaN,NaN
8,1013,"TC_DAYS in 2~6 → TC_DAGE,TC_EC_LT 금지",F03,ForbiddenFilled,"TC_DAYS in[2,3,4,5,6]","must be NULL in [TC_DAGE, TC_EC_LT]",2.0,NaN,1.0,"TC_DAGE,TC_EC_LT",NaN,NaN,NaN,NaN,NaN,NaN
9,1301,"TC_DAYS in 2~6 → TC_DAGE,TC_EC_LT 금지",F03,ForbiddenFilled,"TC_DAYS in[2,3,4,5,6]","must be NULL in [TC_DAGE, TC_EC_LT]",2.0,NaN,2.0,"TC_DAGE,TC_EC_LT",NaN,NaN,NaN,NaN,NaN,NaN



▶ Gate Missing (부모 NaN인데 자식 값 존재): 154110


,_row,_desc,_rule,_type,_condition,_expectation,TC_DAYS,TC_DAGE,TC_AMNT,TC_EC_LT,_viol_cols,TC_EC_MN,TC_EC_DAGE,TC_HTP_LT,TC_HTP_MN,TC_HTP_DAGE,TC_HTP_AMNT,TC_ACCESS,TC_PLC,TC_GET_M,TC_QT_YR
0,0,Q56 missing but children present,G_Q56,GateMissing,TC_DAYS is NaN,parent TC_DAYS is NON-NULL before children [TC...,NaN,NaN,NaN,1.0,"TC_DAGE,TC_AMNT,TC_EC_LT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Q56 missing but children present,G_Q56,GateMissing,TC_DAYS is NaN,parent TC_DAYS is NON-NULL before children [TC...,NaN,NaN,NaN,1.0,"TC_DAGE,TC_AMNT,TC_EC_LT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Q56 missing but children present,G_Q56,GateMissing,TC_DAYS is NaN,parent TC_DAYS is NON-NULL before children [TC...,NaN,NaN,NaN,1.0,"TC_DAGE,TC_AMNT,TC_EC_LT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Q56 missing but children present,G_Q56,GateMissing,TC_DAYS is NaN,parent TC_DAYS is NON-NULL before children [TC...,NaN,NaN,NaN,1.0,"TC_DAGE,TC_AMNT,TC_EC_LT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Q56 missing but children present,G_Q56,GateMissing,TC_DAYS is NaN,parent TC_DAYS is NON-NULL before children [TC...,NaN,NaN,NaN,1.0,"TC_DAGE,TC_AMNT,TC_EC_LT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Q56 missing but children present,G_Q56,GateMissing,TC_DAYS is NaN,parent TC_DAYS is NON-NULL before children [TC...,NaN,NaN,NaN,1.0,"TC_DAGE,TC_AMNT,TC_EC_LT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,Q56 missing but children present,G_Q56,GateMissing,TC_DAYS is NaN,parent TC_DAYS is NON-NULL before children [TC...,NaN,NaN,NaN,1.0,"TC_DAGE,TC_AMNT,TC_EC_LT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Q56 missing but children present,G_Q56,GateMissing,TC_DAYS is NaN,parent TC_DAYS is NON-NULL before children [TC...,NaN,NaN,NaN,2.0,"TC_DAGE,TC_AMNT,TC_EC_LT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,Q56 missing but children present,G_Q56,GateMissing,TC_DAYS is NaN,parent TC_DAYS is NON-NULL before children [TC...,NaN,NaN,NaN,1.0,"TC_DAGE,TC_AMNT,TC_EC_LT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,Q56 missing but children present,G_Q56,GateMissing,TC_DAYS is NaN,parent TC_DAYS is NON-NULL before children [TC...,NaN,NaN,NaN,1.0,"TC_DAGE,TC_AMNT,TC_EC_LT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



총 오류(결측/논리) 개수: 156465


,_row,_desc,_rule,_type,_condition,_expectation,TC_LT,TC_FAGE,_viol_cols,TC_DAYS,TC_DAGE,TC_EC_LT,TC_EC_FAGE,TC_EC_MN,TC_EC_DAGE,TC_HTP_LT,TC_HTP_FAGE,TC_HTP_MN,TC_HTP_DAGE,_group,TC_AMNT,TC_HTP_AMNT,TC_PLC,TC_GET_M,TC_QT_YR,TC_ACCESS
0,603,TC_LT==2 → TC_FAGE 필수,R02,RequiredMissing,TC_LT ==2,must be NON-NULL in [TC_FAGE],2.0,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RequiredMissing,NaN,NaN,NaN,NaN,NaN,NaN
1,792,TC_LT==2 → TC_FAGE 필수,R02,RequiredMissing,TC_LT ==2,must be NON-NULL in [TC_FAGE],2.0,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RequiredMissing,NaN,NaN,NaN,NaN,NaN,NaN
2,1767,TC_LT==2 → TC_FAGE 필수,R02,RequiredMissing,TC_LT ==2,must be NON-NULL in [TC_FAGE],2.0,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RequiredMissing,NaN,NaN,NaN,NaN,NaN,NaN
3,2028,TC_LT==2 → TC_FAGE 필수,R02,RequiredMissing,TC_LT ==2,must be NON-NULL in [TC_FAGE],2.0,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RequiredMissing,NaN,NaN,NaN,NaN,NaN,NaN
4,2338,TC_LT==2 → TC_FAGE 필수,R02,RequiredMissing,TC_LT ==2,must be NON-NULL in [TC_FAGE],2.0,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RequiredMissing,NaN,NaN,NaN,NaN,NaN,NaN
5,3244,TC_LT==2 → TC_FAGE 필수,R02,RequiredMissing,TC_LT ==2,must be NON-NULL in [TC_FAGE],2.0,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RequiredMissing,NaN,NaN,NaN,NaN,NaN,NaN
6,3796,TC_LT==2 → TC_FAGE 필수,R02,RequiredMissing,TC_LT ==2,must be NON-NULL in [TC_FAGE],2.0,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RequiredMissing,NaN,NaN,NaN,NaN,NaN,NaN
7,4249,TC_LT==2 → TC_FAGE 필수,R02,RequiredMissing,TC_LT ==2,must be NON-NULL in [TC_FAGE],2.0,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RequiredMissing,NaN,NaN,NaN,NaN,NaN,NaN
8,4427,TC_LT==2 → TC_FAGE 필수,R02,RequiredMissing,TC_LT ==2,must be NON-NULL in [TC_FAGE],2.0,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RequiredMissing,NaN,NaN,NaN,NaN,NaN,NaN
9,4491,TC_LT==2 → TC_FAGE 필수,R02,RequiredMissing,TC_LT ==2,must be NON-NULL in [TC_FAGE],2.0,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RequiredMissing,NaN,NaN,NaN,NaN,NaN,NaN



규칙별 건수 요약:


,_group,_rule,_desc,_condition,_expectation,count
0,ForbiddenFilled,F03,"TC_DAYS in 2~6 → TC_DAGE,TC_EC_LT 금지","TC_DAYS in[2,3,4,5,6]","must be NULL in [TC_DAGE, TC_EC_LT]",1102
1,ForbiddenFilled,F04,"TC_DAYS==7 → TC_AMNT,TC_EC_LT 금지",TC_DAYS ==7,"must be NULL in [TC_AMNT, TC_EC_LT]",840
2,ForbiddenFilled,F10,TC_HTP_MN==7 → TC_HTP_AMNT 금지,TC_HTP_MN ==7,must be NULL in [TC_HTP_AMNT],192
3,ForbiddenFilled,F11,NOT (TC_DAYS|TC_EC_MN|TC_HTP_MN in 2~7) → TC_P...,(composite) (NOT in 2~7 for ALL of TC_DAYS/TC_...,"must be NULL in [TC_PLC, TC_GET_M, TC_QT_YR]",2
7,GateMissing,G_Q65,Q65 missing but children present,TC_HTP_MN is NaN,parent TC_HTP_MN is NON-NULL before children [...,52638
6,GateMissing,G_Q61,Q61 missing but children present,TC_EC_MN is NaN,parent TC_EC_MN is NON-NULL before children [T...,51028
5,GateMissing,G_Q56,Q56 missing but children present,TC_DAYS is NaN,parent TC_DAYS is NON-NULL before children [TC...,50442
4,GateMissing,G_COMPOSITE,Composite parents missing but children present,"all parents NaN: TC_DAYS, TC_EC_MN, TC_HTP_MN","parents TC_DAYS, TC_EC_MN, TC_HTP_MN should be...",2
8,RequiredMissing,R02,TC_LT==2 → TC_FAGE 필수,TC_LT ==2,must be NON-NULL in [TC_FAGE],87
9,RequiredMissing,R06,TC_DAYS==7 → TC_DAGE 필수,TC_DAYS ==7,must be NON-NULL in [TC_DAGE],32


In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display

THRESH = 0.03  # 팀 기준: 3%

# 안전장치: combined 없으면 재구성 시도
if "combined" not in globals():
    try:
        combined = pd.concat([
            req_df.assign(_group="RequiredMissing"),
            forb_df.assign(_group="ForbiddenFilled"),
            gate_df.assign(_group="GateMissing")
        ], ignore_index=True)
    except Exception as e:
        raise RuntimeError("combined이 없습니다. 먼저 결측/오류 탐지 셀을 실행해 주세요.") from e

# df 존재 확인
try:
    N = len(df)
except NameError:
    raise RuntimeError("df가 없습니다. 먼저 데이터 로딩 셀을 실행해 주세요.")

# 0) 전체 오류율 (오류가 1개라도 있는 행의 비율)
rows_with_any_error = combined["_row"].dropna().astype(int).nunique()
overall_rate = rows_with_any_error / N
print(f"전체 행 수: {N:,}")
print(f"오류가 있는 행 수(중복 제거): {rows_with_any_error:,}  → 전체 오류율: {overall_rate:.2%}")

# 1) 변수별 오류율
tmp = combined.copy()
tmp["_viol_cols"] = tmp["_viol_cols"].fillna("")
tmp["_viol_cols_list"] = tmp["_viol_cols"].apply(
    lambda s: [c.strip() for c in str(s).split(",") if c and c.strip()]
)
vlong = tmp.explode("_viol_cols_list", ignore_index=True)
vlong = vlong[vlong["_viol_cols_list"].notna() & (vlong["_viol_cols_list"] != "")]

per_var = (
    vlong.groupby("_viol_cols_list")["_row"].nunique()
    .reset_index(name="n_rows_flagged")
    .rename(columns={"_viol_cols_list": "column"})
)
per_var["rate"] = per_var["n_rows_flagged"] / N
per_var["recommendation"] = np.where(per_var["rate"] > THRESH, "impute", "drop")
per_var = per_var.sort_values("rate", ascending=False)

print("\n[변수별 오류율 상위 30]")
display(per_var.head(30))

to_impute_vars = per_var.loc[per_var["rate"] > THRESH, "column"].tolist()
to_drop_vars   = per_var.loc[per_var["rate"] <= THRESH, "column"].tolist()
print(f"\n권고 - impute 대상 변수({THRESH:.0%} 초과): {len(to_impute_vars)}개")
print(to_impute_vars[:30])
print(f"권고 - drop 대상 변수({THRESH:.0%} 이하): {len(to_drop_vars)}개")
print(to_drop_vars[:30])

# 2) 규칙별 오류율
per_rule = (
    combined
    .groupby(["_group","_rule","_desc"], dropna=False)["_row"]
    .nunique()
    .reset_index(name="n_rows_flagged")
)
per_rule["rate"] = per_rule["n_rows_flagged"] / N
per_rule["recommendation"] = np.where(per_rule["rate"] > THRESH, "impute", "drop")
per_rule = per_rule.sort_values(["rate","_group"], ascending=[False, True])

print("\n[규칙별 오류율 상위 30]")
display(per_rule.head(30))

# 3) 유형별 요약
per_type = (
    combined.groupby("_group")["_row"].nunique()
    .reset_index(name="n_rows_flagged")
)
per_type["rate"] = per_type["n_rows_flagged"] / N
print("\n[오류 유형별 요약]")
display(per_type)

# ----- 여기까지가 코드 셀입니다 -----
# 아래처럼 '출력 예시' 텍스트를 붙이고 싶다면 주석으로!
# 7 GateMissing G_Q65 Q65 missing but children present 52638 0.963131 impute
# 6 GateMissing G_Q61 Q61 missing but children present 51028 0.933672 impute
# 5 GateMissing G_Q56 Q56 missing but children present 50442 0.922950 impute


전체 행 수: 54,653
오류가 있는 행 수(중복 제거): 54,194  → 전체 오류율: 99.16%

[변수별 오류율 상위 30]


,column,n_rows_flagged,rate,recommendation
6,TC_HTP_AMNT,52830,0.966644,impute
7,TC_HTP_DAGE,52638,0.963131,impute
0,TC_ACCESS,52638,0.963131,impute
4,TC_EC_LT,52384,0.958484,impute
2,TC_DAGE,51544,0.943114,impute
1,TC_AMNT,51282,0.938320,impute
3,TC_EC_DAGE,51028,0.933672,impute
8,TC_HTP_LT,51028,0.933672,impute
5,TC_GET_M,2,0.000037,drop
9,TC_PLC,2,0.000037,drop



권고 - impute 대상 변수(3% 초과): 8개
['TC_HTP_AMNT', 'TC_HTP_DAGE', 'TC_ACCESS', 'TC_EC_LT', 'TC_DAGE', 'TC_AMNT', 'TC_EC_DAGE', 'TC_HTP_LT']
권고 - drop 대상 변수(3% 이하): 3개
['TC_GET_M', 'TC_PLC', 'TC_QT_YR']

[규칙별 오류율 상위 30]


,_group,_rule,_desc,n_rows_flagged,rate,recommendation
7,GateMissing,G_Q65,Q65 missing but children present,52638,0.963131,impute
6,GateMissing,G_Q61,Q61 missing but children present,51028,0.933672,impute
5,GateMissing,G_Q56,Q56 missing but children present,50442,0.922950,impute
0,ForbiddenFilled,F03,"TC_DAYS in 2~6 → TC_DAGE,TC_EC_LT 금지",1102,0.020164,drop
1,ForbiddenFilled,F04,"TC_DAYS==7 → TC_AMNT,TC_EC_LT 금지",840,0.015370,drop
2,ForbiddenFilled,F10,TC_HTP_MN==7 → TC_HTP_AMNT 금지,192,0.003513,drop
8,RequiredMissing,R02,TC_LT==2 → TC_FAGE 필수,87,0.001592,drop
9,RequiredMissing,R06,TC_DAYS==7 → TC_DAGE 필수,32,0.000586,drop
12,RequiredMissing,R13,TC_HTP_LT==2 → TC_HTP_FAGE 필수,32,0.000586,drop
10,RequiredMissing,R08,TC_EC_LT==2 → TC_EC_FAGE 필수,30,0.000549,drop



[오류 유형별 요약]


,_group,n_rows_flagged,rate
0,ForbiddenFilled,1971,0.036064
1,GateMissing,53052,0.970706
2,RequiredMissing,190,0.003476


In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display

# 안전장치
if "df" not in globals() or "combined" not in globals():
    raise RuntimeError("df 또는 combined이 없습니다. 먼저 결측/오류 탐지 셀을 실행해 주세요.")

# 특수코드(응답무효 등) → '존재'로 보지 않음. 필요하면 수정
INVALID_CODES = {0, 96, 97, 98, 99, 999}

def present_1d(obj: pd.Series | pd.DataFrame) -> pd.Series:
    """
    값의 '존재'를 bool Series로 반환.
    - Series면 그대로, DataFrame(동명이인 컬럼 중복)이면 row-wise any.
    - INVALID_CODES는 존재로 보지 않음.
    """
    if isinstance(obj, pd.DataFrame):
        m = obj.apply(lambda s: s.notna() & ~s.isin(INVALID_CODES))
        return m.any(axis=1)
    else:
        return obj.notna() & ~obj.isin(INVALID_CODES)

# 3개 GateMissing 규칙 설정
RULES = {
    "G_Q56": {  # parent: TC_DAYS
        "parent": "TC_DAYS",
        "trigger": "TC_DAGE",                        # 있으면 → TC_DAYS=7 (확정)
        "conflict_children": ["TC_AMNT", "TC_EC_LT"] # AMNT→2~6, EC_LT 응답→1 경로 신호
    },
    "G_Q61": {  # parent: TC_EC_MN
        "parent": "TC_EC_MN",
        "trigger": "TC_EC_DAGE",                     # 있으면 → TC_EC_MN=7 (확정)
        "conflict_children": ["TC_HTP_LT"]           # 1~6이면 HTP_LT로 진행
    },
    "G_Q65": {  # parent: TC_HTP_MN
        "parent": "TC_HTP_MN",
        "trigger": "TC_HTP_DAGE",                    # 있으면 → TC_HTP_MN=7 (확정)
        "conflict_children": ["TC_HTP_AMNT", "TC_ACCESS"]  # AMNT→2~6, ACCESS→1
    },
}

rows = []
for rid, cfg in RULES.items():
    # 이 규칙으로 플래그된 유니크 행
    sub = (combined[(combined["_group"]=="GateMissing") & (combined["_rule"]==rid)]
           .drop_duplicates(subset=["_row"]))
    flagged = sub["_row"].dropna().astype(int).unique()
    if len(flagged) == 0:
        rows.append({
            "rule_id": rid,
            "parent": cfg["parent"],
            "trigger": cfg["trigger"],
            "n_flagged": 0,
            "n_deterministic": 0,
            "n_non_deterministic": 0,
            "deterministic_rate": np.nan
        })
        continue

    # 필요한 컬럼만 추출 (동명이인 컬럼 대비하여 DataFrame로 받아 처리)
    need_cols = [c for c in [cfg["trigger"]] + cfg["conflict_children"] if c in df.columns]
    X = df.loc[flagged, need_cols] if need_cols else df.loc[flagged, []]

    # 확정적: 트리거 '있음' & 어떤 상충 신호도 '없음'
    trig = present_1d(X.filter(items=[cfg["trigger"]])) if cfg["trigger"] in X.columns else pd.Series(False, index=X.index)
    if cfg["conflict_children"]:
        conf_list = []
        for cc in cfg["conflict_children"]:
            if cc in X.columns:
                conf_list.append(present_1d(X.filter(items=[cc])))
        if conf_list:
            # 위치 기반 OR (인덱스 정렬 이슈 방지)
            conf = np.zeros(len(X), dtype=bool)
            for s in conf_list:
                conf |= s.values
            conf = pd.Series(conf, index=X.index)
        else:
            conf = pd.Series(False, index=X.index)
    else:
        conf = pd.Series(False, index=X.index)

    det_mask = trig.values & (~conf.values)  # numpy로 위치기반 결합
    n_total = len(X)
    n_det   = int(det_mask.sum())
    n_nond  = int(n_total - n_det)

    rows.append({
        "rule_id": rid,
        "parent": cfg["parent"],
        "trigger": cfg["trigger"],
        "n_flagged": n_total,
        "n_deterministic": n_det,
        "n_non_deterministic": n_nond,
        "deterministic_rate": (n_det / n_total) if n_total else np.nan
    })

summary_det = pd.DataFrame(rows).sort_values("n_flagged", ascending=False)
print("▶ GateMissing(3개 규칙) 확정적 역추론 가능/불가 집계")
display(summary_det)



▶ GateMissing(3개 규칙) 확정적 역추론 가능/불가 집계


,rule_id,parent,trigger,n_flagged,n_deterministic,n_non_deterministic,deterministic_rate
2,G_Q65,TC_HTP_MN,TC_HTP_DAGE,52638,0,52638,0.0
1,G_Q61,TC_EC_MN,TC_EC_DAGE,51028,0,51028,0.0
0,G_Q56,TC_DAYS,TC_DAGE,50442,0,50442,0.0
